# We collect Stats on the grammar

first initialise logger :) 

In [ ]:
%matplotlib inline
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=1)

In [ ]:
# get data
from eden.converter.graph.gspan import gspan_to_eden
from itertools import islice
def get_graphs(dataset_fname, size=100):
    return  islice(gspan_to_eden(dataset_fname),size)

dataset_fname = 'toolsdata/bursi.pos.gspan'

### induce a grammar and train an estimator


In [ ]:
%%time
from graphlearn.graphlearn import Sampler
training_graphs = get_graphs(dataset_fname, size=200)
sampler=Sampler(radius_list=[0,1],thickness_list=[2],
                          min_cip_count=2, min_interface_count=2)
sampler.fit(training_graphs,grammar_n_jobs=1, grammar_batch_size=10)

print('graph grammar stats:')
n_instances, n_interfaces, n_cores, n_cips = sampler.grammar().size()
print('#instances: %d   #interfaces: %d   #cores: %d   #core-interface-pairs: %d' % (n_instances,
                                                                                     n_interfaces,
                                                                                     n_cores,
                                                                                     n_cips))

### what are the most frequent CIPS?

In [ ]:
#draw production rules
from graphlearn.utils.draw import draw_grammar
draw_grammar(sampler.grammar().productions, contract=True,
             n_productions=6,n_graphs_per_line=6, 
             size=5, colormap='rainbow', node_border=1, vertex_alpha=0.7, edge_alpha=0.5, node_size=700)


### how are the CIPs distributed?

In [ ]:
#grammar statistics
from graphlearn.utils.draw import draw_grammar_stats
draw_grammar_stats(sampler.lsgg.productions, size=(10,5))

### analyse grammar size under different parameters

In [ ]:
%%time
from graphlearn.graphlearn import Sampler
from itertools import tee
for radius in range(0,2):
    for thickness in range(1,3):
        print 'Radius: %d   Thickness: %d' % (radius, thickness)
        for size in range(100,250,50):
            training_graphs = get_graphs(dataset_fname, size=size)
            training_graphs,training_graphs_=tee(training_graphs)

            sampler=Sampler(radius_list=[radius],
                                      thickness_list=[thickness],
                                      min_cip_count=2, 
                                      min_interface_count=2,
                                      random_state=42)

            sampler.fit(training_graphs_,grammar_n_jobs=1, grammar_batch_size=10)

            n_instances, n_interfaces, n_cores, n_cips = sampler.lsgg.size()
            print('#instances: %3d   #interfaces: %4d   #cores: %4d   #core-interface-pairs: %5d' % (n_instances,
                                                                                                 n_interfaces,
                                                                                                 n_cores,
                                                                                                 n_cips))

---